In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [4]:
cwd = os.getcwd()
print(cwd)

/cluster/home/skyriakos/chemprop_run/git/notebooks


In [5]:
os.chdir('..')
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.run_workflow import *
from deepadr.chemfeatures import *
from deepadr.hyphelperflat import *
# from deepadr.model_gnn import GCN as testGCN
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN
# from deepadr.model_attn_siamese import *
from ogb.graphproppred import Evaluator
os.chdir(cwd)

In [6]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [7]:
# from tdc.single_pred import Tox
# from tdc.multi_pred import DDI

In [8]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [9]:
report_available_cuda_devices()

number of GPUs available: 1
cuda:0, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB



In [10]:
n_gpu = torch.cuda.device_count()
n_gpu

1

In [11]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

# fdtype = torch.float32

In [12]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 1.9.1
CUDA: 11.1
3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:41:03) 
[GCC 9.4.0]


In [13]:
# print(torch.cuda.memory_summary(device=device_gpu))

### Preparing dataset 

In [14]:
score = 'total_thresh' #'total_thresh'
score_val = 4

In [15]:
# v_1: GNN
# v_2: Alt Models

In [16]:
# TDC Tox
# DSdataset_name = 'SynergxZloewe' #'OncoPolyPharmacology' #'DrugComb'
DSdataset_name = f'DrugComb_{score}_{score_val}' #'DrugComb'


#fname_suffix = ds_config["fname_suffix"]
# similarity_types = ['chem']
# kernel_option = 'sqeuclidean'
data_fname = 'data_v2'
# interact_matfname = ds_config["interact_matfname"]
# exp_iden = 'simtypeall'
# ddi_interaction_labels_pth = ds_config["ddi_interaction_labels_pth"]

# up_dir, processed_dir, DSdataset_name, data_fname

In [17]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
/cluster/home/skyriakos/chemprop_run/git/data/processed/DrugComb_total_thresh_4/data_v2


In [18]:
xFlat = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'X_flat.pkl'))
y = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'y.pkl'))
expression = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'expression.pkl'))

In [19]:
xFlatMat = torch.stack([torch.cat(i) for i in list(xFlat.values())])
xFlatMat.shape

torch.Size([25757, 18])

In [20]:
torch.tensor(expression)

tensor([[ 0.1661,  0.2518,  1.3373,  ..., -0.4085,  0.8581,  0.8979],
        [ 0.1661,  0.2518,  1.3373,  ..., -0.4085,  0.8581,  0.8979],
        [ 0.1661,  0.2518,  1.3373,  ..., -0.4085,  0.8581,  0.8979],
        ...,
        [-0.0373, -1.4322,  1.5411,  ..., -1.2711,  1.2013,  0.9537],
        [-0.0373, -1.4322,  1.5411,  ..., -1.2711,  1.2013,  0.9537],
        [-0.0373, -1.4322,  1.5411,  ..., -1.2711,  1.2013,  0.9537]],
       dtype=torch.float64)

In [21]:
x = torch.cat([xFlatMat, torch.tensor(expression)], dim=1)
x.shape

torch.Size([25757, 926])

In [22]:
# %%time

# # Make sure to first run the "data_generation" notebook first

# dataset = MoleculeDataset(root=targetdata_dir)

In [23]:
# print()
# print(f'Dataset: {dataset}:')
# print('====================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# # data0 = dataset[0]  # Get the first graph object.

# # print()
# # print(data)
# # print('=============================================================')

# # # Gather some statistics about the first graph.
# # print(f'Number of nodes: {data.num_nodes}')
# # print(f'Number of edges: {data.num_edges}')
# # print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
# # print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
# # print(f'Contains self-loops: {data.contains_self_loops()}')
# # print(f'Is undirected: {data.is_undirected()}')

In [24]:
# data0

In [25]:
# data0.expression.shape

In [26]:
# data0.y

In [27]:
# dataset.tensors[1]

In [28]:
# used_dataset = dataset

# If you want to use a smaller subset of the dataset for testing
# smaller_dataset_len = int(len(dataset)/1)
# used_dataset = dataset[:smaller_dataset_len]

In [29]:
# type(used_dataset)

In [30]:
# dataset.data.y

In [31]:
y

array([0, 0, 0, ..., 0, 1, 1])

In [32]:
x.shape

torch.Size([25757, 926])

In [33]:
y.shape

(25757,)

In [34]:
# fold_partitions = get_stratified_partitions(dataset.data.y,
#                                             num_folds=5,
#                                             valid_set_portion=0.1,
#                                             random_state=42)

In [35]:
fold_partitions = get_stratified_partitions(y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.5764356969533567
class: 1 norm count: 0.4235643030466433
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5763929334109882
class: 1 norm count: 0.4236070665890118

-------------------------
fold_num: 3
train data
class: 0 norm count: 0.576

In [36]:
# train_val_test_frac = [0.7, 0.1, 0.2]
# assert sum(train_val_test_frac) == 1

# torch.manual_seed(42)
# used_dataset = used_dataset.shuffle()

# num_train = round(train_val_test_frac[0] * len(used_dataset)) 
# num_trainval = round((train_val_test_frac[0]+train_val_test_frac[1]) * len(used_dataset))

# train_dataset = used_dataset[:num_train]
# val_dataset = used_dataset[num_train:num_trainval]
# test_dataset = used_dataset[num_trainval:]

In [37]:
# train_dataset = Subset(used_dataset, fold_partitions[0]['train'])
# val_dataset = Subset(used_dataset, fold_partitions[0]['validation'])
# test_dataset = Subset(used_dataset, fold_partitions[0]['test'])

In [38]:
# print(f'Number of training graphs: {len(train_dataset)}')
# print(f'Number of val graphs: {len(val_dataset)}')
# print(f'Number of test graphs: {len(test_dataset)}')

In [39]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))


Number of training graphs: 18544
Number of validation graphs: 2061
Number of testing graphs: 5152


### Preprocessing

In [40]:
# import functools

# def compose(*functions):
#     return functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [41]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [42]:
x_np = x.numpy()
y_np = y

In [43]:
pipeline = compose(scaler.fit_transform, np.tanh, scaler.fit_transform)

In [44]:
x_np_norm = pipeline(x_np)

In [45]:
deepsynergy_input_size = x_np_norm.shape[1]
deepsynergy_input_size

926

In [46]:
ids = list(range(len(y)))

In [47]:
dataset = TensorDataset(torch.tensor(x_np_norm),torch.tensor(y), torch.tensor(ids))
dataset

### Deep Synergy

In [48]:
# print(dataset.num_classes)

In [49]:
# training params total_thresh 4
tp = {
    "batch_size" : 300,
    "num_epochs" : 100,
    
    "emb_dim" : 300,
    "gnn_type" : "gatv2",
    "num_layer" : 5,
    "graph_pooling" : "mean", #attention
    
    "input_embed_dim" : None,
    "gene_embed_dim": 1,
    "num_attn_heads" : 2,
    "num_transformer_units" : 1,
    "p_dropout" : 0.3,
#     "nonlin_func" : nn.ReLU(),
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-5, #3e-4
    "max_lr_mul": 5,
    "l2_reg" : 1e-5,
    "loss_w" : 1.,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 908,
    "exp_H1" : 500,
    "exp_H2" : 400
}

In [50]:
tp['deepsynergy_input_size'] = deepsynergy_input_size
tp['deepsynergy_input_size']

926

In [51]:
# # training params total_thresh 3
# tp = {
#     "batch_size" : 300,
#     "num_epochs" : 200,
    
#     "emb_dim" : 300,
#     "gnn_type" : "gatv2",
#     "num_layer" : 5,
#     "graph_pooling" : "mean", #attention
    
#     "input_embed_dim" : None,
#     "gene_embed_dim": 1,
#     "num_attn_heads" : 2,
#     "num_transformer_units" : 1,
#     "p_dropout" : 0.3,
# #     "nonlin_func" : nn.ReLU(),
#     "mlp_embed_factor" : 2,
#     "pooling_mode" : 'attn',
#     "dist_opt" : 'cosine',

#     "base_lr" : 3e-5, #3e-4
#     "max_lr_mul": 10,
#     "l2_reg" : 1e-8,
#     "loss_w" : 0.3,
#     "margin_v" : 1.,

#     "expression_dim" : 64,
#     "expression_input_size" : 908,
#     "exp_H1" : 500,
#     "exp_H2" : 400
# }

In [52]:
# targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# targetdata_dir_exp

In [53]:
# device_gpu = get_device(True, index=0)
# exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp0"))
# tphp = generate_tp_hp(tp, hyperparam_space[0], hp_names)
# partition = fold_partitions[0]

In [54]:
# run_exp(tphp, device_gpu, exp_dir,partition=fold_partitions[0], used_dataset=used_dataset)

In [55]:
def spawn_q_process(q_process):
    print(">>> spawning hyperparam search process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined hyperparam search process")
    
def create_q_process(queue, used_dataset, gpu_num, tphp, exp_dir, partition): #
#     fold_gpu_map = {0:gpu_num}
    return mp.Process(target=deepadr.hyphelperflat.run_exp_flat, args=(queue, used_dataset, gpu_num, tphp, exp_dir, partition)) #

In [56]:
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

queue = mp.Queue()
q_processes = []

# partition = fold_partitions[0]
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

print("Start: " + time_stamp)

for q_i in range(min(n_gpu, len(fold_partitions))):
#     device_gpu = get_device(True, index=q_i)
    partition = fold_partitions[q_i]
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))

#     tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)
    
    q_process = create_q_process(queue, dataset, q_i, tp, exp_dir, partition)
    q_processes.append(q_process)
    spawn_q_process(q_process)

spawned_processes = n_gpu
    
# for q_i in range(len(hyperparam_space)):
for q_i in range(min(n_gpu, len(fold_partitions))):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    print("released_gpu_num:", released_gpu_num)
#     if(spawned_processes < len(hyperparam_space)):
# #         device_gpu = get_device(True, index=q_i)
#         time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
#         exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp_"+str(q_i)+"_"+time_stamp))
#         tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)

#         q_process = create_q_process(queue, used_dataset, released_gpu_num, tphp, exp_dir, partition)
#         q_processes.append(q_process)
#         spawn_q_process(q_process)
#         spawned_processes = spawned_processes + 1

Start: 2022-03-24_13-45-20
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
gpu: cuda:0
DS model:
 DeepSynergy(
  (fc1): Linear(in_features=926, out_features=8192, bias=True)
  (fc2): Linear(in_features=8192, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=2, bias=True)
  (drop_in): Dropout(p=0.2, inplace=False)
  (drop): Dropout(p=0.5, inplace=False)
  (log_softmax): LogSoftmax(dim=-1)
)
=====Epoch 0
Training...


Iteration:  19%|█▉        | 12/62 [00:00<00:00, 119.36it/s]

Evaluating...


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

{'Train':  best_epoch_indx:0
 auc:0.7293280735789168 
 apur:0.6880282193232747 
 f1:0.6098853691606064 
 precision:0.5910882809700155 
 recall:0.6299172501591344 
, 'Validation':  best_epoch_indx:0
 auc:0.7293280735789168 
 apur:0.6880282193232747 
 f1:0.6098853691606064 
 precision:0.5910882809700155 
 recall:0.6299172501591344 
, 'Test':  best_epoch_indx:0
 auc:0.7293280735789167 
 apur:0.6880282193402694 
 f1:0.6098853691606064 
 precision:0.5910882809700155 
 recall:0.6299172501591344 
}
=====Epoch 1
Training...


...

{'Train':  best_epoch_indx:98
 auc:0.960882443440698 
 apur:0.949726738735913 
 f1:0.8701251366784108 
 precision:0.8321133960729639 
 recall:0.9117759388924253 
, 'Validation':  best_epoch_indx:98
 auc:0.960882443440698 
 apur:0.949726738735913 
 f1:0.8701251366784108 
 precision:0.8321133960729639 
 recall:0.9117759388924253 
, 'Test':  best_epoch_indx:98
 auc:0.960882443440698 
 apur:0.949726738735913 
 f1:0.8701251366784108 
 precision:0.8321133960729639 
 recall:0.9117759388924253 
}
=====Epoch 99
Training...


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

Evaluating...


Iteration: 100%|██████████| 62/62 [00:00<00:00, 141.27it/s]


{'Train':  best_epoch_indx:99
 auc:0.965055945781248 
 apur:0.9550283097230822 
 f1:0.8763741325308603 
 precision:0.8465828191741813 
 recall:0.9083386378103119 
, 'Validation':  best_epoch_indx:99
 auc:0.965055945781248 
 apur:0.9550283097230822 
 f1:0.8763741325308603 
 precision:0.8465828191741813 
 recall:0.9083386378103119 
, 'Test':  best_epoch_indx:99
 auc:0.965055945781248 
 apur:0.9550283097230822 
 f1:0.8763741325308603 
 precision:0.8465828191741813 
 recall:0.9083386378103119 
}
Finished training!
<<< joined hyperparam search process
released_gpu_num: 0


In [57]:
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))


End: 2022-03-24_13-51-08
